In [138]:
import pandas as pd
import numpy as np
from scipy import stats

**This Python code focuses on cleaning HVAC - related merged data. It first reads in the 
merged data and converts the 'TIMESTAMP' column to datetime. Then, it performs a series of 
cleaning steps including preprocessing special values, handling missing values (using 
interpolation for numeric columns and filling with a specific string for text columns), and 
detecting outliers with Z - score method. Finally, it saves the cleaned data.**

In [139]:
# Load previously merged data
final_data = pd.read_csv('../../outputs/HVAC_merged_data.csv')
final_data['TIMESTAMP'] = pd.to_datetime(final_data['TIMESTAMP'])

C:\Users\12503\AppData\Local\Temp\ipykernel_65824\759342792.py:2: DtypeWarning: Columns (25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,65,66,67,68,69,70,71,72,73,74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  final_data = pd.read_csv('../../outputs/HVAC_merged_data.csv')


In [140]:
def general_data_cleaning(df):
    df, text_columns = preprocess_special_values(df)
    # Missing value processing
    df = handle_missing_values(df)

    # outlier detection
    df = detect_outliers(df, text_columns)
    
    return df


def preprocess_special_values(df):
    print("\n=== Special Character Preprocessing ===")
    special_values = ['-', '--', '---', 'NA', 'N/A', 'NULL', 'NaN', '']
    text_columns = ['source_file_y', 'source_file_x']  
    
    for col in df.columns:
        if col in text_columns or col == 'TIMESTAMP':  
            continue
            
        if df[col].dtype == 'object':
            print(f"Trying to convert column {col} to a numeric type...")
            try:
                df[col] = df[col].replace(special_values, np.nan)
                df[col] = pd.to_numeric(df[col], errors='coerce')  
                print(f"  Successfully converted column {col} to a numeric type")
            except Exception as e:
                print(f"  Conversion failed, remain as text column: {str(e)}")
                text_columns.append(col)
        else:
            text_columns.append(col)

    # Processing text columns
    for col in text_columns:
        if col in df.columns:
            print(f"Processing text columns {col}")
            df[col] = df[col].replace(special_values, 'UNKNOWN')
            df[col] = df[col].astype(str)  # Make sure it's a string type

    return df, text_columns

def detect_outliers(df, text_columns):
    print("\n=== outlier detection ===")
    # Detecting outliers in numeric columns using the Z-score method (excluding text and time columns)
    numeric_cols = [col for col in df.select_dtypes(include=['float64', 'int64']).columns 
                   if col not in text_columns and col != 'TIMESTAMP']
    
    for col in numeric_cols:
        # Ensure that there are no infinitely large or small values in the columns
        df[col] = df[col].replace([np.inf, -np.inf], np.nan)
        
        # Calculate Z-score
        try:
            valid_values = df[col].dropna()
            if len(valid_values) < 2:  # 2 less values are needed to calculate the Z-score
                print(f"Column {col} Insufficient valid values, skip outlier detection")
                continue
                
            z_scores = np.abs(stats.zscore(valid_values))
            
            # Defining outlier thresholds
            threshold = 3
            outliers_mask = np.abs(stats.zscore(df[col].fillna(df[col].median()))) > threshold
            outliers = df[col][outliers_mask]
            
            if not outliers.empty:
                print(f"Column {col} found {len(outliers)} outlier (Z-score > {threshold})")
                # Replace with median
                median_val = df[col].median()
                df.loc[outliers_mask, col] = median_val
        except Exception as e:
            print(f"Unable to compute an outlier for column {col}.: {str(e)}")
    
    return df


def handle_missing_values(df):
    print("\n=== Missing value handling ===")
    # Recognize text columns
    text_columns = [col for col in df.columns 
                   if col.startswith('source_file_') or 
                      (df[col].dtype == 'object' and not pd.api.types.is_numeric_dtype(df[col]))]
    
    # Calculate the proportion of missing values in each column (excluding text columns)
    missing_stats = df.drop(columns=text_columns, errors='ignore').isnull().mean().sort_values(ascending=False)
    print("Statistics on the percentage of missing values in numeric columns:")
    print(missing_stats[missing_stats > 0])
    
    # Use linear interpolation for a small number of missing values in a numeric column
    threshold = 0.1  # 10% as a threshold
    for col in df.columns:
        if col in text_columns or col == 'TIMESTAMP':
            continue
            
        if pd.api.types.is_numeric_dtype(df[col]):
            if df[col].isnull().mean() < threshold:
                df[col] = df[col].interpolate(method='linear')
                print(f"The numeric column {col} uses linear interpolation to fill in missing values.")
            elif df[col].isnull().mean() >= threshold:
                print(f"Numeric columns {col} High percentage of missing values ({df[col].isnull().mean():.2%})")
    
    # Handling missing values in text columns
    for col in text_columns:
        if col in df.columns and df[col].isnull().any():
            missing_count = df[col].isnull().sum()
            df[col] = df[col].fillna('MISSING')
            print(f"The text column {col} is populated with {missing_count} with a missing value of 'MISSING'")
    
    return df

In [141]:
# Execution of the cleaning process
print("Begin the data cleansing process...")
cleaned_data = general_data_cleaning(final_data)
# cleaned_data = hvac_specific_cleaning(cleaned_data)


# Save cleaned data
cleaned_data.to_csv('../../outputs/HVAC_cleaned_data.csv', index=False)
print("\nData cleansing is complete and the results have been saved to ../../outputs/HVAC_cleaned_data.csv")

# Display of information on cleaned data
print("\nPost-cleaning data information.")
display(cleaned_data.info())
display(cleaned_data.head())


Begin the data cleansing process...

=== Special Character Preprocessing ===
Trying to convert column T_Sup_RTU to a numeric type...
  Successfully converted column T_Sup_RTU to a numeric type
Trying to convert column RH_Sup_RTU to a numeric type...
  Successfully converted column RH_Sup_RTU to a numeric type
Trying to convert column T_Ret_RTU to a numeric type...
  Successfully converted column T_Ret_RTU to a numeric type
Trying to convert column RH_Ret_RTU to a numeric type...
  Successfully converted column RH_Ret_RTU to a numeric type
Trying to convert column T_VAV_102 to a numeric type...
  Successfully converted column T_VAV_102 to a numeric type
Trying to convert column T_VAV_103 to a numeric type...
  Successfully converted column T_VAV_103 to a numeric type
Trying to convert column T_VAV_104 to a numeric type...
  Successfully converted column T_VAV_104 to a numeric type
Trying to convert column T_VAV_105 to a numeric type...
  Successfully converted column T_VAV_105 to a nume

None

,TIMESTAMP,T_Stair_101,T_Room_102,T_Room_103,T_Room_104,T_Room_105,T_Room_106,T_Stair_201,T_Room_202,T_Room_203,...,source_file_x,T_out,RH_out,BP,Dir_Solar,Dif_Solar,Glo_Solar,WS,WD,source_file_y
0,2021-02-23 00:00:00,10.78888889,15.58333333,20.42222222,16.40555556,19.25,17.84444444,12.98888889,18.90555556,16.38888889,...,Building_FF_Heating.csv,1.25,89.8,98377.8156,0.106,0.092,-1.358,0.0,0.0,Weather_FF_Heating.csv
1,2021-02-23 00:01:00,10.78333333,15.58333333,20.38888889,16.40555556,19.23333333,17.81666667,12.98333333,18.93333333,16.38888889,...,Building_FF_Heating.csv,1.266666667,89.5,98377.8156,0.053,0.092,-1.409,0.0,0.0,Weather_FF_Heating.csv
2,2021-02-23 00:02:00,10.78333333,15.56111111,20.42222222,16.38888889,19.24444444,17.83333333,12.97222222,18.91111111,16.41111111,...,Building_FF_Heating.csv,1.3,89.1,98377.8156,0.106,0.092,-1.358,0.0,0.0,Weather_FF_Heating.csv
3,2021-02-23 00:03:00,10.78333333,15.57777778,20.38333333,16.39444444,19.24444444,17.80555556,12.97222222,18.93333333,16.36111111,...,Building_FF_Heating.csv,1.361111111,88.5,98377.8156,0.106,0.138,-1.409,0.0,0.0,Weather_FF_Heating.csv
4,2021-02-23 00:04:00,10.78333333,15.54444444,20.41111111,16.38888889,19.23333333,17.82777778,12.96111111,18.93333333,16.39444444,...,Building_FF_Heating.csv,1.35,88.3,98377.8156,0.053,0.092,-1.409,0.0,0.0,Weather_FF_Heating.csv
